# Polychromatic PSF simulations with EFC
This notebook demonstrates how to simulate polychromatic PSFs with the scoobPSF package and run EFC. <br>
It follows from the scoob_psf_demo_poly notebook and the scoob_model_efc notebook. <br>
Note that there are several "FIXME's" that are included for discussion on how we might re-structure this package.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import astropy.units as u
from astropy.io import fits
from pathlib import Path
from IPython.display import clear_output, display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from importlib import reload
import os
import copy

import poppy
import ray
import lina

import logging, sys
poppy_log = logging.getLogger('poppy')
poppy_log.setLevel('DEBUG')
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
poppy_log.disabled = True

import scoobpsf
from scoobpsf.math_module import xp, _scipy
from scoobpsf.imshows import *
from scoobpsf import scoobm

## Optical System Parameters
The following cell provides parameters of the optical system

In [ ]:
pupil_diam = 6.75*u.mm 
lyot_diam=3.6*u.mm
det_rotation = 0 #  degrees of rotation of the detector relative to the optical axis

# Flat wavefront as determined by Kyle doing phase diversity measurements.
dm_flat = fits.getdata(scoobm.module_path/'scoob_dm_flat.fits')
dm_flat0=copy.copy(dm_flat)
# Known bad actuator is 26,21 - useful to keep this value 
# as a variable since "dm_flat" evolves throughout the notebook
bad_act_value=dm_flat[26,21]

## Simulation Parameters
The following cell provides parameters which are specific to the simulation

In [ ]:
# wavelength_c = 632.8e-9*u.m # central wavelength
wavelength_c = 630e-9*u.m # central wavelength
npix=int(512/4)  # Sampling of pupil plane?
oversample=int(16/4)
npsf = int(400)  # Side length of the camera in pixels
use_opds=True #  Incorporate WFE from each optical component

### Set wavelength sampling parameters
Note that the bandwidth is to be set in nm, but is provided below as a percentage of the central wavelength.
The parallelization of the simulation follows from the number of wavelengths (nlam.

In [ ]:
bandwidth=0.05*wavelength_c #nm
bandwidth=10*u.nm
nlam = 11 # number of wavelengths, which is also the number of actors

In [ ]:
minlam = wavelength_c - bandwidth/2
maxlam = wavelength_c + bandwidth/2
wavelengths = np.linspace( minlam, maxlam, nlam )
print(f'PSF will be built using wavelengths: {wavelengths.to("nm")}')

### Set source flux for each wavelength
Array will be normalized so the integrated flux is 1.

In [ ]:
# Set Source Flux for each wavelength
# Assume a flat spectrum for now 
f_lambda=xp.ones(len(wavelengths))

f_lambda/=xp.sum(f_lambda)
print(f_lambda)

## Declare the coronograph setup
Note that by default, the coronograph is not installed when the class gets instantiated.

In [ ]:
vortex = scoobpsf.agpm.IdealAGPM(name='VVC', wavelength=wavelength_c, charge=6, rotation=20)
lyot_stop = poppy.CircularAperture(name='Lyot Stop', radius=lyot_diam/2.0)

#### Optional: Use the knife edge as a FPM
Knife edge shifted to 2 lambda/D.<br>
Note that he f/# at the FPM is 48.

In [ ]:
shift = (2 * wavelength_c * 48).to("um")
knife_edge = poppy.KnifeEdge(name='Knife Edge', rotation=0, shift_x=-shift)

## Declare bad actuators

In [ ]:
# bad_acts=0 # No bad actuators
bad_acts=[(26,21)] # 1 dead actuator

## Initialize the class with the appropriate parameters for a non-coronographic PSF

In [ ]:
scoob_kwargs = {'npix':npix, 
                'npsf':npsf,
                'bad_acts': bad_acts,
                'oversample':oversample, 
                'det_rotation':det_rotation, 
                'use_opds':use_opds,
                'pupil_diam':pupil_diam,
}

In [ ]:
reload(scoobm)
model = scoobm.SCOOBM(**scoob_kwargs)

Make a ray actor class from the original scoobm class 

In [ ]:
ray_scoobm = ray.remote(scoobm.SCOOBM)

Instantiate one class (actor) per wavelength

In [ ]:
actors = []
for i in range(nlam):
    actors.append(ray_scoobm.options(num_gpus=1/nlam).remote(**scoob_kwargs))
    # Set actor specific keywords
    actors[i].setattr.remote('wavelength', wavelengths[i])

Instantiate the parallelizedScoob class which handles the actors

In [ ]:
# This is just for debugging convenience
reload(scoobm)
from scoobpsf.scoobm import ParallelizedScoob
p_scoob=ParallelizedScoob(actors,f_lambda)

In [ ]:
# FIXME: Add parameters that are derived in the model but not elsewhere and 
# are needed at the parallelized level
# p_scoob.set_actor_attr('dm_mask',model.dm_mask)
p_scoob.dm_mask=model.dm_mask
p_scoob.dm_bad_act_mask = model.dm_bad_act_mask
p_scoob.Nact=model.Nact
p_scoob.psf_pixelscale_lamD = model.psf_pixelscale_lamD

## Add desired shape on the DM

In [ ]:
# The flattest wavefront obtained for the system in the lab by
# performing phase diversity
# dm_flat = fits.getdata(scoobm.module_path/'scoob_dm_flat.fits')

In [ ]:
p_scoob.set_dm(dm_flat)

### Pin the bad actuators to desired value(s)


In [ ]:
pin=True
if pin:
    # pinned_value=med_val+(np.sqrt(49.)*(bad_act_value-med_val)/16)
    pinned_value=bad_act_value
    if bad_acts:
        print(f'Pinning {len(bad_acts)} bad actuators at {pinned_value*1e9:0.3f} nm')
        for act in bad_acts:
            # print(act)
            dm_flat[act]=pinned_value

## Run all wavelengths unocculted to determine normalization factors
Setting the actor attributes to None first ensures the correct masks are in which is helpful if cells are not run sequentially.

In [ ]:
fpm=None
lyot=None
p_scoob.set_actor_attr('FPM',fpm)
p_scoob.set_actor_attr('LYOT',lyot)

psfs_unocc = p_scoob.snaps()
# imshow1(psfs_unocc[0], f'Image for wavelength {wavelengths[0].to("nm"):0.1f}', lognorm=True)

### First insert FPM & Lyot Mask to determine the occulted PSFs

In [ ]:
# fpm = knife_edge
fpm = vortex
lyot = lyot_stop

In [ ]:
# p_scoob.set_actor_attr('FPM',vortex)
p_scoob.set_actor_attr('FPM',fpm)
p_scoob.set_actor_attr('LYOT',lyot)

In [ ]:
# DO NOT NORMALIZE - this will be done later
# The reason for not normalizing here is due to the future weighting of the relative fluxes for each wavelength
# Note that it could probably still be done here, but I wonder if it's better to do a re-think on this.

### Set normalization factor based on the maximum of the unocculted value
# Setting this here doesn't work as expected when running EFC.
# Or maybe this is because I need to recalculate the jacobian.
# for i in range(nlam):
    # actors[i].setattr.remote('im_norm', psfs_unocc[i].max())

Create array of unocculted and non-normalized PSFs

In [ ]:
psfs_occ = p_scoob.snaps()

Display a slice

In [ ]:
i=0
norm=psfs_unocc[i].max()
imshow1(psfs_occ[i]/norm, f'Normalized Image for wavelength {wavelengths[0].to("nm"):0.1f}', lognorm=True)

## Define the Region of the Dark Hole

In [ ]:
reload(lina.utils)
npsf = int(npsf)  # called npsf in scoobm
nact = 34  # FIXME: currently hard coded in scoobm

xfp = (xp.linspace(-npsf/2, npsf/2-1, npsf) + 1/2)*model.psf_pixelscale_lamD
fpx,fpy = xp.meshgrid(xfp,xfp)

edge = 2
iwa = 3
owa = 10
rot = 0

# Create the mask that is used to select which region to make dark.
dark_params = {
    'inner_radius' : iwa,
    'outer_radius' : owa,
    'edge' : edge,
    'rotation':rot,
}
dark_mask = lina.utils.create_annular_focal_plane_mask(fpx, fpy, dark_params)
imshow2(dark_mask, dark_mask*psfs_occ[i], "dark mask", "occulted image * mask",
        lognorm2=True)

## Perform intensity weighting and combination of PSFs
This section creates unocculted and occulted PSFs based on the desired weighting at each wavelength. <br>
The spectrum is normalized such that the integral over the wavelength range will be 1.

In [ ]:
psf_occ_calib, psf_unocc_calib = ParallelizedScoob.flux_calibrate_psf(psfs_occ, f_lambda, norm=psfs_unocc)

Need pixel scale for display. <br>
The pixelscale of 4.63um corresponds to ~0.2 lam/D per pixel using the 150mm final imaging lens. <br>
The value was determined empirically by injecting sinusoidal waves. <br>
FIXME: this is hard coded in the model but should come up a few levels to the instrument configuration level

In [ ]:
psf_pixelscale = 4.63e-6*u.m/u.pix
psf_pixelscale_lamD = (1/(5)) * psf_pixelscale.to(u.m/u.pix).value/4.63e-6

In [ ]:
imshow2(psf_unocc_calib, psf_occ_calib, lognorm1=True, lognorm2=True, pxscl=model.psf_pixelscale_lamD)

In [ ]:
# Calculate the base metrics for the DH
print(f'Total counts in DH: {np.sum(dark_mask*psf_occ_calib):0.3e}')
print(f'Mean value in DH: {np.mean(dark_mask*psf_occ_calib):0.3e}')
print(f'Contrast in DH: {np.std(dark_mask*psf_occ_calib):0.3e}')

## Calculate the complex wavefront (this is for debugging)
This requires scaling first or the jacobian will be wrong.

In [ ]:
# this will provide a complex wavefront
poly_psf = p_scoob.calc_psf()

In [ ]:
norm=psf_unocc_calib.max()
print(f'{norm=}')
inten=xp.abs((poly_psf/np.sqrt(norm))**2)
imshow1(inten, f'Normalized Image for wavelength {wavelengths[0].to("nm"):0.1f}', 
        lognorm=True,vmin=1e-8,vmax=1)

In [ ]:
# # Calculate the base metrics for the DH
print(f'Total counts in DH: {np.sum(dark_mask*inten):0.3e}')
print(f'Mean value in DH: {np.mean(dark_mask*inten):0.3e}')
print(f'Contrast in DH: {np.std(dark_mask*inten):0.3e}')

In [ ]:
reload(lina.efc)

p_scoob.set_dm(dm_flat)
epsilon = 1e-9 # poke amplitudes (DM surface), presumably in meters

### Derive and save the Jacobian 
This calculation only needs to be done once per system setup, so saving it is useful.<br>
FIXME: We need a new filename system, plus the information on system setup should go into the fits header.
One way to do this would be to derive the filename of the jacobian early on, before any simulations are run. <br>
The resulting simulated values can then go into a dictionary that is then passed on and modified in each step.

In [ ]:
if bool(bad_acts):
    filename=f'jac-efc-npix{model.npix}-oversample{model.oversample}-fpm{fpm.name}-cw{wavelength_c.to("nm").value:0.0f}-bp{bandwidth.to("nm").value:0.0f}-nlam{nlam}-badacts{len(bad_acts)}.fits'.replace(" ","")
else:
    filename=f'jac-efc-npix{model.npix}-oversample{model.oversample}-fpm{fpm.name}-cw{wavelength_c.to("nm").value:0.0f}-bp{bandwidth.to("nm").value:0.0f}-nlam{nlam}.fits'.replace(" ","")
print(f'{filename=}')

In [ ]:
if os.path.isfile(filename) is False:
    jac = lina.efc.build_jacobian(p_scoob, epsilon, dark_mask, plot=False)
    scoobpsf.utils.save_fits(filename, scoobpsf.math_module.ensure_np_array(jac))
else:
    print('Jacobian exists, skipping')

In [ ]:
jac = xp.array(fits.getdata(filename))

In [ ]:
reload(lina.utils)
response = lina.utils.map_acts_to_dm(xp.sqrt(((jac)**2).sum(axis=0)).get(), model.dm_mask)
imshow1(response, lognorm=True, vmin=1e3, vmax=3e6)

### Normalization discussion
FIXME: The response here is a factor of 7.73 lower than the monochromatic simulations. <br>
Looks to be because calc_psfs isn't normalizing by the non-occulted wavefront. <br>
I think we might want to consider how the normalization is structured in the scoobPSF code as it might be worth abstracting to perform at the parallelization layer.

In [ ]:
# np.median(response[model.dm_mask])

## Run EFC

In [ ]:
reload(lina.efc)
reload(lina.utils)
reload(lina.math_module)
model.set_dm(dm_flat)

In [ ]:
%time
# declare penalty matrix value, and number of iterations for each value (10)
# -4 --> -1 represent the beta value for regularization

# Sidick starts at -4, then goes down to -1
reg_conds = [(-4, 10), (-3,10), (-2,10), (-1, 10)]

for i in range(len(reg_conds)):
    print(f'{i=}')
    # Derive the control matrix, which is the gain matrix from Sidick 2012
    # details are described in utils.beta_reg.
    # matrix is then flattened, therefore not in 2d and matched to dm coords.
    control_matrix = lina.utils.beta_reg(jac, reg_conds[i][0])
    
    # Assume a system with perfect knowledge of the E-field
    ims, commands, sms_fig = lina.efc.run_efc_perfect(p_scoob, 
                                            jac, 
                                            control_matrix,
                                            dark_mask, 
                                            Imax_unocc=1,
                                            efc_loop_gain=0.5, 
                                            iterations=reg_conds[i][1], 
                                            plot_all=True, 
                                            plot_sms=False,
                                            plot_radial_contrast=True)

## Normalize the Contrast Values
This is just done by hand for now until the proper solution is discussed and implemented

In [ ]:
2.126e-12/(psf_unocc_calib.max())

## Plot final DM shape
show final DM shape, with the flat removed. <br>
This shows how the DM moved as a result of the EFC runs.

In [ ]:
tmp=(p_scoob.get_dm()-dm_flat)*p_scoob.dm_mask
imshow1(tmp, 'DM shape - flat')

# Re-calculate metrics for DH
med_val = np.median(tmp[model.dm_mask])
std_val = np.std(tmp[model.dm_mask])
ptv=np.max(tmp[model.dm_mask])-np.min(tmp[model.dm_mask])
print(f'Median is: {med_val*1e9:0.1f} nm')
print(f'stddev is: {std_val*1e9:0.1f} nm')
print(f'PtV is: {ptv*1e9:0.1f} nm')